# Evaluation of Object Detection

## Average Precision

> The implementation of Average Precision is mainly based on the 11-point Interpolated Average Precision that is proposed in the book 'Introduction to Information Retrieval', https://nlp.stanford.edu/IR-book/html/htmledition/evaluation-of-ranked-retrieval-results-1.html. 

The implementation of the Average Precision here is adapted from the 101 Point Interpolation AP evaluation metric that is introduced by MS COCO in 2014.
- AP is calculated at 101 points rather than 11 points, on ranging recall values from 0 to 1 at a step of 0.01 
- AP is calculated for a set of 10 different IoU thresholds and then averaged. The IoU threshold ranges from 0.5 to 0.95 at a step frequency of 0.05

## Average Recall

> The implementation of Average Recall is mainly adapted from the paper 'What makes for effective detection proposals?' https://arxiv.org/pdf/1502.05082.pdf. 
- AR is calculated as 2 times area under the recall-iou curve
- Recall is calculated for 5 different IoU thresholds starting from 0.5 to 1 at a step frequency of 0.1

---

Some of the considerations for the evaluation is explained below:
- Unmatched detecting bounding boxes are considered FP
- Matched detecting bounding boxes with IoU overlap greater than or equal to IoU threshold are considered TP 
- **Precision** is calculated as TP / (TP + FP)
- TP + FP is calculated as the number of predicted bounding boxes
- **Recall** is calculated as TP / (TP + FN)
- TP + FN is considered as the total number of ground truth bounding boxes for dataset
- Detected bounding boxes with confidence score less than score threshold are not included in the evaluation

In [ ]:
def calc_iou(ground_truth_bbox, pred_bbox):
    x1, y1, x2, y2 = ground_truth_bbox
    p_x1, p_y1, p_x2, p_y2 = pred_bbox
    
    # determine possible intersection points
    left_max = x1 if x1 > p_x1 else p_x1
    right_min = x2 if p_x2 > x2 else p_x2
    top_max = y1 if y1 > p_y1 else p_y1
    bottom_min = y2 if p_y2 > y2 else p_y2
    
    # return 0 if bounding boxes do not intersect 
    if left_max > right_min or top_max > bottom_min:
        return 0.0
    
    # compute width and height of overlapping area
    overlap_x = right_min - left_max
    overlap_y = bottom_min - top_max

    # compute intersection area       
    intersect_area = overlap_x * overlap_y
    
    # compute union area = area of gt_bbox + area of pred_bbox - intersection area
    w1, h1, w2, h2 = x2 - x1, y2 - y1, p_x2 - p_x1, p_y2 - p_y1  
    union_area = (w1 * h1) + (w2 * h2) - intersect_area
    
    # compute iou = intersection area / area of union
    iou = intersect_area / float(union_area)

    return iou

In [ ]:
# A detected BB (BBdt) and a ground truth BB (BBgt)
# form a potential match if they overlap sufficiently (iou >= .5)

# each BBdt and BBgt may be matched at most once
# detections with highest confidence are matched first

# match detected bounding boxes with ground-truth bounding boxes 
def get_iou_of_matched_bbox(gt_bbox_list, dt_bbox_list, iou_threshold = 0.5):
    
    len_dt_bbox = dt_bbox_list.shape[0]
    len_gt_bbox = gt_bbox_list.shape[0]
    
    if len_dt_bbox == 0:
        return []
    
    iou_matrix = np.zeros((len_dt_bbox, len_gt_bbox))
    
    # for each detection, match with highest overlap  
    matched_bboxes = []
    used_ind = []
    for i, dt_bbox in enumerate(dt_bbox_list):   # detections are already sorted by confidence scores
        for j, gt_bbox in enumerate(gt_bbox_list):
            if not j in used_ind:   # if ground-truth bbox is available
                
                # calculate iou
                iou_matrix[i][j] = calc_iou(gt_bbox, dt_bbox)
        
        # get the highest overlap
        idx = np.argmax(iou_matrix[i,:])
        
        # check if bounding boxes form a potential match (iou >= .5)
        if iou_matrix[i][idx] >= iou_threshold:
            # add the matched bboxes indices with iou
            matched_bboxes.append([i, idx, iou_matrix[i][idx]])
            # remove gorund-truth bbox from the matrix, set availability
            used_ind.append(idx)
            
    return np.array(matched_bboxes)

In [ ]:
# Average Precision: 101 point interpolation AP of precision-recall curve
def calc_average_precision(eval_table, iou_thresholds, pred_scores, total_num_gt_bbox):
    
    # init a list of average precision values for different iou thresholds
    ap_vals = np.zeros(iou_thresholds.shape)

    # 101 recall points for interpolation
    recall_vals = np.around(np.arange(0, 1.01, 0.01), decimals=2)    # [0:.01:1] R=101
    
    # idx of confidence scores in descending order
    score_idx_list = np.argsort(pred_scores)[::-1]
    
    for i, iou_th in enumerate(iou_thresholds):
        # get true positive values of detections
        tp = np.array(eval_table[iou_th])
        
        # sort predictions based on confidence scores in descending order
        tp = tp[score_idx_list]

        # get false positives
        fp = 1 - tp

        # get accumulated tp and fp
        acc_tp = np.cumsum(tp)
        acc_fp = np.cumsum(fp)

        # calculate precision and recall
        precision = acc_tp / (acc_tp + acc_fp)
        recall = acc_tp / total_num_gt_bbox
    
        
        # get interpolated precision values on recall points with constraint that
        # highest precision found for any recall level r' >= r
        
        # get maximum precision vals of reversed precision values
        # reverse the result for recall vals
        inter_p = np.maximum.accumulate(precision[::-1])[::-1]

        # populate recall indices
        recall_idx = np.searchsorted(recall, recall_vals, side="left")

        # get 101 point interpolated precision values
        inter_p_101 = np.array([inter_p[r] if r < len(inter_p) else 0 for r in recall_idx])
        
        # compute average precision
        ap = np.mean(inter_p_101)
        
        ap_vals[i] = ap
        
        # if iou is .5 or .75 
        # print out AP50 and AP75 values
        if iou_th == 0.5 or iou_th == 0.75:
            print('AP{}: {}'.format(int(iou_th * 100), np.around(ap * 100, decimals=2)))
    
    # get mean of average precision values
    ap = np.mean(ap_vals)
    
    return ap

In [ ]:
# compute Average Recall: 2 * area under the recall-iou curve    
def calc_average_recall(eval_table, iou_vals, total_num_gt_bbox):
    
    # init recall values for different iou thresholds
    recall = np.zeros(iou_vals.shape)
    
    for i, iou_val in enumerate(iou_vals):
        # get true positive info of detections
        tp = np.array(eval_table[iou_val])

        # get total number of true positives
        total_tp = np.sum(tp)
        
        # calculate recall
        recall[i] = total_tp / total_num_gt_bbox
    
    # compute average recall -> 2 * area under the recall-iou curve
    ar = 2 * np.trapz(recall, iou_vals)    # trapz - integrate y(x) along each 1d slice on the given axis
    
    return ar

In [ ]:
# confidence score threshold  
score_threshold = 0.45

# iou thresholds for average precision, AP -> [.50:.05:.95]
ap_iou_thresholds = np.around(np.arange(0.5, 1, 0.05), decimals=2)

# iou values for average recall, AR -> [.50:.1:1]
ar_iou_thresholds = np.around(np.arange(0.5, 1.01, 0.1), decimals=2)

# iou thresholds for AP and AR
iou_thresholds = np.union1d(ap_iou_thresholds, ar_iou_thresholds)

In [ ]:
def eval():
    # set model to evaluation mode
    model.eval()
    
    # evaluation table
    # in form of { iou_threshold: [ tp ] }
    eval_dict = {}    
                        
    
    # for different iou thresholds record tp info of each detection  
    for iou_th in iou_thresholds:
        eval_dict[iou_th] = []
    
    total_num_gt_bbox = 0
    
    all_pred_scores = [] # list of confidence scores of detections

    # unmatched BBdt count as false positives and unmatched BBgt as false negatives
    # matched BBdt with iou >= iou_thresold is considered true positives

    for images, labels in data_loader:
        with torch.no_grad():
            predictions = model(images)
        
        # for each image in test set
        for pred, label in zip(predictions, labels):
            # get gorund-truth bounding boxes
            gt_bboxes = label['boxes']
            
            # get detected bounding boxes and scores
            # filter predictions based on a confidence score threshold
            pred_bboxes = pred['boxes'][pred['scores'] > score_threshold]
            pred_scores = pred['scores'][pred['scores'] > score_threshold] 
                        
            # increase total number of ground truth bboxes
            total_num_gt_bbox += len(gt_bboxes)
            
            # match bounding boxes for gt and dt and get ious
            # in form of (dt_idx, gt_idx, iou)
            iou_bboxes = get_iou_of_matched_bbox(gt_bboxes, pred_bboxes)
            
            # if no matched bbox is found, preds are fp
            # record pred tp, pred score and continue on new image
            if len(iou_bboxes) == 0: 
                tp = 0
                for pred_score in pred_scores:
                    for iou_th in iou_thresholds:
                        eval_dict[iou_th].append(tp) 
                    all_pred_scores.append(pred_score)
                continue
                    
            # get matched dt inds and ious
            ious = iou_bboxes[:, 2]
            matched_dt_inds = iou_bboxes[:, 0]
            
            # for each prediction
            for j, (pred_bbox, pred_score) in enumerate(zip(pred_bboxes, pred_scores)):
                # check if prediction is unmatched
                fp = 0 if j in matched_dt_inds else 1
                if fp == 1:
                    # record tp value on each iou threshold
                    for iou_th in iou_thresholds:
                        eval_dict[iou_th].append(1 - fp)
            
                else:
                    # get iou overlap for the detection
                    iou, ious = ious[0], ious[1:]
                    # for a set of different iou thresholds
                    for iou_th in iou_thresholds:    
                        # compute tp
                        tp = 1 if iou >= iou_th else 0
                        # record detection tp value
                        eval_dict[iou_th].append(tp)
                
                # add confidence score of detection to the score list  
                all_pred_scores.append(pred_score)
                
        
    # get average precision
    ap = calc_average_precision(eval_dict, ap_iou_thresholds, all_pred_scores, total_num_gt_bbox)
    
    # calculate average recall
    ar = calc_average_recall(eval_dict, ar_iou_thresholds, total_num_gt_bbox)
    
    print('AP: {}'.format(np.around(ap * 100, decimals=2)))
    print('AR: {}'.format(np.around(ar * 100, decimals=2)))

    return ap, ar
    


In [ ]:
eval()

AP50: 16.11
AP75: 7.42
AP: 8.16
AR: 12.84


(0.0816240605645483, 0.12844638949671772)

## Explanations

 
### Evaluation

The evaluation happens by recording True Positive value of the each detected bounding box for the given IoU threshold. By forming an evaluation table at each IoU threshold, TP value and confidence score of every detection is recorded. The values needed to be calculated for Average Precision and Average Recall evaluation can be derived from TP value of detections, confidence score ranking and total number of ground truth bounding boxes.

#### Algorithm

for each image in test set:<br>
&emsp;get ground truth bounding boxes<br>
&emsp;get detected bounding boxes<br>
&emsp;match dt with gt bounding box and get IoU values<br>
&emsp;for each detected bounding box:<br>
&emsp;&emsp;for each IoU threshold:<br>
&emsp;&emsp;&emsp;if detection is unmatched(FP) record 0<br>
&emsp;&emsp;&emsp;if detection is matched but IoU overlap is less than IoU threshold(FP) record 0<br> 
&emsp;&emsp;&emsp;if detection is matched and IoU overlap is greater than or equal to IoU threshold(TP) record 1<br>
&emsp;&emsp;record confidence score<br>
calculate Average Precision with recorded values<br>
calculate Average Recall with recorded values

---

Above implementation considers only bounding boxes for evaluation and omits classification.

#### Some Helpful Resources

- What is Average Precision in Object Detection & Localization Algorithms and how to calculate it?, https://towardsdatascience.com/what-is-average-precision-in-object-detection-localization-algorithms-and-how-to-calculate-it-3f330efe697b
- A Survey on Performance Metrics for Object-Detection Algorithms, https://github.com/rafaelpadilla/Object-Detection-Metrics/blob/master/paper_survey_on_performance_metrics_for_object_detection_algorithms.pdf
- Precision, Recall & Mean Average Precision for Object Detection Playlist, https://www.youtube.com/playlist?list=PL1GQaVhO4f_jE5pnXU_Q4MSrIQx4wpFLM
- COCO Dataset Detection Evaluation, https://cocodataset.org/#detection-eval
- An Introduction to Evaluation Metrics for Object Detection Blog Post, https://blog.zenggyu.com/en/post/2018-12-16/an-introduction-to-evaluation-metrics-for-object-detection/#fn3